<div align="center">
  <h1>Data Storing</h1>
  <h2>Data Hunian Rumah dan Apartemen di Jakarta Pusat</h2>
</div>

- Install dan import library

In [1]:
# # jika packages berikut belum terinstall, un-comment dan jalankan cell ini untuk menginstallnya
# %pip install --upgrade pip
# %pip install mysqlclient --quiet

# note: jika ada yang belum terinstall lakukan command di cell baru:
# %pip install <nama package>

In [2]:
import MySQLdb, os, json, subprocess

- Definisi user dbms dan load data json yang telah di-preprocess

In [3]:
__HOST__        = "localhost"
__USER__        = "root"
__PASSWORD__    = "1234"
__DATABASE__    = "property"

__RED__     = '\033[91m'
__GREEN__   = '\033[92m'
__RESET__   = '\033[0m'

__CURRENT_DIRECTORY__   = os.getenv('CURRENT_DIR', os.getcwd())
__PATH_FOLDER__         = os.path.abspath(os.path.join(__CURRENT_DIRECTORY__, '..', 'data'))

perusahaan_path =  os.path.join(__PATH_FOLDER__, 'perusahaan.json')
agen_path       =  os.path.join(__PATH_FOLDER__, 'agen.json')
hunian_path     =  os.path.join(__PATH_FOLDER__, 'hunian.json')
rumah_path      =  os.path.join(__PATH_FOLDER__, 'rumah.json')
apartemen_path  =  os.path.join(__PATH_FOLDER__, 'apartemen.json')

with open(perusahaan_path)  as file: perusahaan_data = json.load(file)
with open(agen_path)        as file: agen_data = json.load(file)
with open(hunian_path)      as file: hunian_data = json.load(file)
with open(rumah_path)       as file: rumah_data = json.load(file)
with open(apartemen_path)   as file: apartemen_data = json.load(file)

- Membuat basis data dan schema tabel basis data

In [4]:
drop_database   = f"DROP DATABASE IF EXISTS {__DATABASE__};"
create_database = f"CREATE DATABASE IF NOT EXISTS {__DATABASE__};"
setchar         = f"ALTER DATABASE {__DATABASE__} CHARACTER SET = utf8mb4 COLLATE = utf8mb4_unicode_ci;"
use_database    = f"USE {__DATABASE__}"

perusahaan_schema = """ 
CREATE TABLE IF NOT EXISTS `perusahaan` (
    `nama_perusahaan` VARCHAR(100),
    `alamat` VARCHAR(255) NOT NULL,
    PRIMARY KEY(`nama_perusahaan`)
)
"""

agen_schema = """
CREATE TABLE IF NOT EXISTS `agen` (
    `id_agen` INT NOT NULL,
    `nama_agen` VARCHAR(255) NOT NULL,
    `nomor_telepon` BIGINT NOT NULL,
    `terjual` INT NOT NULL,
    `tersewa` INT NOT NULL,
    `nama_perusahaan` VARCHAR(100),
    PRIMARY KEY (`id_agen`),
    FOREIGN KEY (`nama_perusahaan`) REFERENCES `perusahaan`(`nama_perusahaan`) ON DELETE CASCADE
)
"""

hunian_schema = """
CREATE TABLE IF NOT EXISTS `hunian` (
    `id_iklan` VARCHAR(20) NOT NULL,
    `tipe_properti` ENUM('Rumah', 'Apartemen') NOT NULL,
    `luas_bangunan` INT NOT NULL,
    `kamar_tidur` INT NOT NULL,
    `kamar_mandi` INT NOT NULL,
    `lokasi` VARCHAR(50) NOT NULL,
    `sertifikat` VARCHAR(50) NOT NULL,
    `tipe_iklan` ENUM('jual', 'sewa') NOT NULL,
    `periode_kepemilikan` ENUM('Pemilik', 'Tahunan', 'Bulanan', 'Harian') NOT NULL,
    `harga` BIGINT NOT NULL,
    `diperbarui` DATE NOT NULL,
    `id_agen` INT NOT NULL,
    PRIMARY KEY (`id_iklan`),
    FOREIGN KEY(`id_agen`) REFERENCES `agen`(id_agen) ON DELETE CASCADE
)
"""

rumah_schema = """
CREATE TABLE IF NOT EXISTS `rumah` (
    `id_iklan` VARCHAR(20) NOT NULL,
    `luas_tanah` INT NOT NULL,
    `carport` INT NOT NULL,
    `taman` ENUM('Ya', 'Tidak') NOT NULL,
    PRIMARY KEY (`id_iklan`),
    FOREIGN KEY(`id_iklan`) REFERENCES `hunian`(`id_iklan`) ON DELETE CASCADE
)
"""

apartemen_schema = """
CREATE TABLE IF NOT EXISTS `apartemen` (
    `id_iklan` VARCHAR(20) NOT NULL,
    `kondisi_properti` VARCHAR(20) NOT NULL,
    `kondisi_perabotan` VARCHAR(20) NOT NULL,
    PRIMARY KEY (`id_iklan`),
    FOREIGN KEY(`id_iklan`) REFERENCES `hunian`(`id_iklan`) ON DELETE CASCADE
)
"""

customer_schema = """
CREATE TABLE IF NOT EXISTS `customer` (
    `id_customer` INT NOT NULL,
    `nama_customer` VARCHAR(255) NOT NULL,
    `nomor_telepon` BIGINT NOT NULL,
    `pekerjaan` VARCHAR(255) NOT NULL,
    `pendapatan` BIGINT NOT NULL,
    PRIMARY KEY (`id_customer`)
)
"""

membeli_schema = """
CREATE TABLE IF NOT EXISTS `membeli` (
    `id_customer` INT NOT NULL,
    `id_iklan` VARCHAR(20) NOT NULL,
    `waktu` DATETIME NOT NULL,
    `metode_pembayaran` VARCHAR(255) NOT NULL,
    PRIMARY KEY (`id_customer`, `id_iklan`),
    FOREIGN KEY (`id_customer`) REFERENCES `customer`(`id_customer`) ON DELETE CASCADE,
    FOREIGN KEY (`id_iklan`) REFERENCES `hunian`(`id_iklan`) ON DELETE CASCADE
)
"""

- Insert atau update data hasil scraping ke tabel schema

In [5]:
# melihat data dari fetch cursor MySQLdb
def show_instance(instance):
    print("\n".join(str(row) for row in instance))

insert_perusahaan = """
INSERT INTO `perusahaan` (nama_perusahaan, alamat) 
    VALUES (%s, %s) 
    ON DUPLICATE KEY UPDATE 
        alamat = VALUES(alamat);
"""

insert_agen = """
INSERT INTO `agen` 
    (id_agen, nama_agen, nomor_telepon, terjual, tersewa, nama_perusahaan) 
    VALUES (%s, %s, %s, %s, %s, %s) 
    ON DUPLICATE KEY UPDATE 
        nama_agen = VALUES(nama_agen), 
        nomor_telepon = VALUES(nomor_telepon), 
        terjual = VALUES(terjual), 
        tersewa = VALUES(tersewa), 
        nama_perusahaan = VALUES(nama_perusahaan);
"""

insert_hunian = """
INSERT INTO `hunian` 
    (id_iklan, tipe_properti, luas_bangunan, kamar_tidur, kamar_mandi, lokasi,
      sertifikat, tipe_iklan, periode_kepemilikan, harga, diperbarui, id_agen) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) 
    ON DUPLICATE KEY UPDATE 
        tipe_properti = VALUES(tipe_properti),
        luas_bangunan = VALUES(luas_bangunan),
        kamar_tidur = VALUES(kamar_tidur),
        kamar_mandi = VALUES(kamar_mandi),
        lokasi = VALUES(lokasi),
        sertifikat = VALUES(sertifikat),
        tipe_iklan = VALUES(tipe_iklan),
        periode_kepemilikan = VALUES(periode_kepemilikan),
        harga = VALUES(harga),
        diperbarui = VALUES(diperbarui),
        id_agen = VALUES(id_agen);
"""

insert_rumah = """
INSERT INTO `rumah` 
    (id_iklan, luas_tanah, carport, taman) 
    VALUES (%s, %s, %s, %s) 
    ON DUPLICATE KEY UPDATE 
        luas_tanah = VALUES(luas_tanah),
        carport = VALUES(carport),
        taman = VALUES(taman);
"""

insert_apartemen = """
INSERT INTO `apartemen` 
    (id_iklan, kondisi_properti, kondisi_perabotan) 
    VALUES (%s, %s, %s) 
    ON DUPLICATE KEY UPDATE 
        kondisi_properti = VALUES(kondisi_properti),
        kondisi_perabotan = VALUES(kondisi_perabotan);
"""

- Connect to MySQLdb dan memasukkan query schema tabel

In [6]:
conn = MySQLdb.connect(
        host        =__HOST__,
        user        =__USER__,
        password    =__PASSWORD__
    )

cursor = conn.cursor()
cursor.execute(drop_database)
cursor.execute(create_database)
cursor.execute(setchar)
cursor.execute(use_database)
cursor.execute(perusahaan_schema)
cursor.execute(agen_schema)
cursor.execute(hunian_schema)
cursor.execute(rumah_schema)
cursor.execute(apartemen_schema)
cursor.execute(customer_schema)
cursor.execute(membeli_schema)

conn.commit()

- Insert data dari hasil scraping ke tabel

In [7]:
for item in perusahaan_data:
    data = (
        item['nama_perusahaan'],
        item['alamat']
    )
    cursor.execute(insert_perusahaan, data)
cursor.execute(""" SELECT * FROM `perusahaan` LIMIT 10""")
conn.commit()

# cek apakah sudah tersimpan
show_instance(cursor)

(' Brighton Akasia Cengkareng, Jakarta Barat', 'Komplek Perum Citra 2 Ext Blok BD 1 A Nomor 8, Kel. Pegadungan, Kec. Kalideres, Kota Jakarta Barat')
(' Brighton Priority Sunter, Jakarta Utara', 'Ruko Sunter Paradise, Blok K No.33, Sunter Agung, Kec. Tj. Priok')
(' ERA Mulia Kelapa Cengkir Branch', 'Jl. Kelapa Gading Boulevard Blok WA2 No.9 , Kelapa Gading')
(' EZ Land ', 'Ruko Taman Cemara Blok U6 No 11, Harapan Indah , Bekasi.')
(' Icon Property PIK', 'Ruko Golf island Boulevard blok a nomor 71')
(' IPROPER', 'Jalan Casablanca, Menteng Dalam, Tebet, Jakarta Selatan')
(' Luxury home property', 'Jl Swadaya Raya,Duren Sawit,Jakarta Timur\t')
(' Max Properti JGC', 'La Seine F5 no 110. Jakarta garden city  Cakung - Jakarta Timur')
(' Stars Realty', 'Ruko Graha Boulevard Summarecon Serpong, Jln Gading Serpong Boulevard BVA1, Curug Sangereng, Kec. Klp. Dua, Tangerang, Banten 15810')
('Abie Property', 'Citilofts Sudirman Unit 22. 02, Jl. KH. Mas Mansyur no. 121 Jakarta Pusat')


In [8]:
for item in agen_data:
    data = (
        item['id_agen'],
        item['nama_agen'],
        item['nomor_telepon'],
        item['terjual'],
        item['tersewa'],
        item['nama_perusahaan']
    )            
    cursor.execute(insert_agen, data)
cursor.execute(""" SELECT * FROM `agen` LIMIT 10""")
conn.commit()

# cek apakah sudah tersimpan
show_instance(cursor)

(231, 'JIMMY SARAGIH', 6281317585282, 11, 15, 'bintang gemilang')
(383, 'Budi Helmi', 622183783588, 6, 1, 'Graha2000')
(438, 'Lidia Suhendra', 628121306688, 0, 0, None)
(542, 'Rudy Cen', 6281574800080, 0, 1, 'FRAN RUIZ Property')
(545, 'Wira Sinaga', 62217198012, 0, 3, 'RE/MAX DESTINY')
(567, 'Wienny Novita', 62816814779, 138, 899, 'XP & Co')
(794, 'Lisa Hafrizal', 6281213000233, 138, 0, 'ERA Star')
(1004, 'Isni ', 6281380802121, 21, 54, 'IS PRO')
(1332, 'Rustam Chen', 62816933583, 10, 5, 'MYPRO')
(1666, 'Riniwati Yakaria', 628161842149, 0, 0, 'Giovanna Realty')


In [9]:
for item in hunian_data:
    data = (
        item['id_iklan'],
        item['tipe_properti'],
        item['luas_bangunan'],
        item['kamar_tidur'],
        item['kamar_mandi'],
        item['lokasi'],
        item['sertifikat'],
        item['tipe_iklan'],
        item['periode_kepemilikan'],
        item['harga'],
        item['diperbarui'],
        item['id_agen']
    )
    cursor.execute(insert_hunian, data)
cursor.execute(""" SELECT * FROM `hunian` LIMIT 10""")
conn.commit()

# cek apakah sudah tersimpan
show_instance(cursor)

('apr1487673', 'Apartemen', 94, 4, 3, 'Cempaka Putih', 'SHM - Sertifikat Hak Milik', 'sewa', 'Tahunan', 70000000, datetime.date(2024, 3, 19), 567567)
('apr1519894', 'Apartemen', 33, 2, 1, 'Cempaka Putih', 'Strata', 'sewa', 'Harian', 4700000, datetime.date(2024, 2, 20), 36837)
('apr1519922', 'Apartemen', 33, 2, 1, 'Cempaka Putih', 'Strata', 'sewa', 'Bulanan', 5000000, datetime.date(2024, 2, 20), 36837)
('apr1546666', 'Apartemen', 33, 2, 1, 'Cempaka Putih', 'HGB - Hak Guna Bangunan', 'sewa', 'Bulanan', 4500000, datetime.date(2024, 7, 29), 75980)
('apr1549701', 'Apartemen', 73, 1, 1, 'Kemayoran', 'SHM - Sertifikat Hak Milik', 'sewa', 'Tahunan', 80000000, datetime.date(2024, 7, 20), 62713)
('apr1550418', 'Apartemen', 34, 2, 1, 'Cempaka Putih', 'HGB - Hak Guna Bangunan', 'sewa', 'Tahunan', 18000000, datetime.date(2024, 7, 29), 75980)
('apr1556324', 'Apartemen', 21, 1, 1, 'Cempaka Putih', 'HGB - Hak Guna Bangunan', 'sewa', 'Tahunan', 13000000, datetime.date(2024, 7, 29), 75980)
('apr1559700'

In [10]:
for item in rumah_data:
    data = (
        item['id_iklan'],
        item['luas_tanah'],
        item['carport'],
        item['taman']
    )
    cursor.execute(insert_rumah, data)
cursor.execute(""" SELECT * FROM `rumah` LIMIT 10""")
conn.commit()

# cek apakah sudah tersimpan
show_instance(cursor)

('hor10047873', 192, 0, 'Ya')
('hor10138471', 1360, 10, 'Tidak')
('hor10143630', 185, 0, 'Ya')
('hor10279272', 626, 1, 'Ya')
('hor10358695', 661, 2, 'Ya')
('hor10410810', 821, 4, 'Tidak')
('hor10509468', 75, 1, 'Ya')
('hor10678524', 225, 1, 'Ya')
('hor10701720', 1000, 2, 'Ya')
('hor10703127', 1360, 2, 'Ya')


In [11]:
for item in apartemen_data:
    data = (
        item['id_iklan'],
        item['kondisi_properti'],
        item['kondisi_perabotan']
    )
    cursor.execute(insert_apartemen, data)
cursor.execute(""" SELECT * FROM `apartemen` LIMIT 10""")
conn.commit()

# cek apakah sudah tersimpan
show_instance(cursor)

('apr1487673', 'Bagus', 'Semi Furnished')
('apr1519894', 'unknown', 'unknown')
('apr1519922', 'unknown', 'unknown')
('apr1546666', 'Bagus', 'Furnished')
('apr1549701', 'Bagus', 'Furnished')
('apr1550418', 'Bagus', 'Unfurnished')
('apr1556324', 'Bagus', 'Unfurnished')
('apr1559700', 'Bagus', 'Furnished')
('apr1576433', 'unknown', 'unknown')
('apr1582096', 'Baru', 'Furnished')


- Constraints check

In [12]:
cursor.execute("""
ALTER TABLE `perusahaan`
    ADD CONSTRAINT IF NOT EXISTS `nonempty_alamat` CHECK (LENGTH(`alamat`) > 0);
""")
conn.commit()

cursor.execute("""
ALTER TABLE `agen`
    ADD CONSTRAINT IF NOT EXISTS `length_telp_agen` CHECK (LENGTH(CAST(`nomor_telepon` AS CHAR)) BETWEEN 5 AND 15);
""")
conn.commit()

cursor.execute("""
ALTER TABLE `agen`
    ADD CONSTRAINT IF NOT EXISTS `posint_terjual` CHECK (`terjual` >= 0);
""")
conn.commit()

cursor.execute("""
ALTER TABLE `agen`
    ADD CONSTRAINT IF NOT EXISTS `posint_tersewa` CHECK (`tersewa` >= 0);
""")
conn.commit()

cursor.execute("""
ALTER TABLE `hunian`
    ADD CONSTRAINT IF NOT EXISTS `posint_luas_bangunan` CHECK (`luas_bangunan` > 0);
""")
conn.commit()

cursor.execute("""
ALTER TABLE `hunian`
    ADD CONSTRAINT IF NOT EXISTS `count_kamar_tidur` CHECK (`kamar_tidur` >= 0);
""")
conn.commit()

cursor.execute("""
ALTER TABLE `hunian`
    ADD CONSTRAINT IF NOT EXISTS `count_kamar_mandi` CHECK (`kamar_mandi` >= 0);
""")
conn.commit()

cursor.execute("""
ALTER TABLE `hunian`
    ADD CONSTRAINT IF NOT EXISTS `posint_harga` CHECK (`harga` >= 0);
""")
conn.commit()

cursor.execute("""
ALTER TABLE `rumah`
    ADD CONSTRAINT IF NOT EXISTS `posint_luas_tanah` CHECK (`luas_tanah` > 0);
""")
conn.commit()

cursor.execute("""
ALTER TABLE `rumah`
    ADD CONSTRAINT IF NOT EXISTS `posint_carport` CHECK (`carport` >= 0);
""")
conn.commit()

cursor.execute("""
ALTER TABLE `customer`
    ADD CONSTRAINT IF NOT EXISTS `length_telp_customer` CHECK (LENGTH(CAST(`nomor_telepon` AS CHAR)) BETWEEN 8 AND 15);
""")
conn.commit()

cursor.execute("""
ALTER TABLE `customer`
    ADD CONSTRAINT IF NOT EXISTS `posint_pendapatan` CHECK (`pendapatan` >= 0);
""")
conn.commit()

cursor.execute("""
ALTER TABLE `customer`
    ADD CONSTRAINT `unique_nomor_telepon_customer` UNIQUE(`nomor_telepon`);
""")
conn.commit()

- Triggers

In [13]:
drop_trigger_diperbarui_date = """DROP TRIGGER IF EXISTS trigger_diperbarui_date;"""
trigger_diperbarui_date = """
CREATE TRIGGER trigger_diperbarui_date
BEFORE UPDATE ON hunian
FOR EACH ROW
BEGIN
    IF NEW.diperbarui < OLD.diperbarui THEN
        SIGNAL SQLSTATE '45000' 
        SET MESSAGE_TEXT = 'Tanggal diperbarui tidak boleh lebih kecil dari tanggal sebelumnya.';
    END IF;
END;
"""

drop_trigger_insert_rumah = """DROP TRIGGER IF EXISTS trigger_insert_rumah;"""
trigger_insert_rumah = """
CREATE TRIGGER trigger_insert_rumah
BEFORE INSERT ON rumah
FOR EACH ROW
BEGIN
    DECLARE v_tipe_properti ENUM('Rumah', 'Apartemen');
    SELECT tipe_properti INTO v_tipe_properti FROM hunian WHERE id_iklan = NEW.id_iklan;
    IF v_tipe_properti <> 'Rumah' THEN
        SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = 'Cannot insert into rumah: tipe_properti must be Rumah';
    END IF;
END;
"""

drop_trigger_insert_apartemen = """DROP TRIGGER IF EXISTS trigger_insert_apartemen;"""
trigger_insert_apartemen = """
CREATE TRIGGER trigger_insert_apartemen
BEFORE INSERT ON apartemen
FOR EACH ROW
BEGIN
    DECLARE v_tipe_properti ENUM('Rumah', 'Apartemen');
    SELECT tipe_properti INTO v_tipe_properti FROM hunian WHERE id_iklan = NEW.id_iklan;
    IF v_tipe_properti <> 'Apartemen' THEN
        SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = 'Cannot insert into apartemen: tipe_properti must be Apartemen';
    END IF;
END;
"""

drop_trigger_update_agen = """DROP TRIGGER IF EXISTS trigger_update_agen;"""
trigger_update_agen = """
CREATE TRIGGER trigger_update_agen
AFTER INSERT ON membeli
FOR EACH ROW
BEGIN
    DECLARE v_tipe_iklan ENUM('jual', 'sewa');

    SELECT tipe_iklan INTO v_tipe_iklan
    FROM hunian
    WHERE id_iklan = NEW.id_iklan;

    IF v_tipe_iklan = 'jual' THEN
        UPDATE agen
        INNER JOIN hunian ON agen.id_agen = hunian.id_agen
        SET agen.terjual = agen.terjual + 1
        WHERE hunian.id_iklan = NEW.id_iklan;
    END IF;

    IF v_tipe_iklan = 'sewa' THEN
        UPDATE agen
        INNER JOIN hunian ON agen.id_agen = hunian.id_agen
        SET agen.tersewa = agen.tersewa + 1
        WHERE hunian.id_iklan = NEW.id_iklan;
    END IF;
END
"""

- Execute query triggers

In [14]:
cursor.execute(drop_trigger_diperbarui_date)
conn.commit()
cursor.execute(trigger_diperbarui_date)
conn.commit()
cursor.execute(drop_trigger_insert_rumah)
conn.commit()
cursor.execute(trigger_insert_rumah)
conn.commit()
cursor.execute(drop_trigger_insert_apartemen)
conn.commit()
cursor.execute(trigger_insert_apartemen)
conn.commit()
cursor.execute(drop_trigger_update_agen)
conn.commit()
cursor.execute(trigger_update_agen)
conn.commit()

- Close MySQLdb

In [15]:
cursor.close()
conn.close()

- Dump database

In [16]:
def dump_database(user, password, db_name, output_file):
    data_storing_folder = os.path.join(__CURRENT_DIRECTORY__, '..', '..', 'Data Storing\\export')
    if not os.path.exists(data_storing_folder):
        os.makedirs(data_storing_folder)

    full_output_file = os.path.join(data_storing_folder, output_file)
    escaped_output_file = f'"{full_output_file}"'
    command = f"mysqldump -u {user} -p{password} {db_name} > {escaped_output_file}"
    
    try:
        subprocess.run(command, shell=True, check=True)
        log_message = (
            f'{__RED__}............................{__RESET__}\n'
            f'............................\n'
            f'{__GREEN__}............................{__RESET__}\n'
            f'{__GREEN__}Data storing has been completed!{__RESET__}\n\n'
            f"Database {db_name} has been dumped to:\n '{full_output_file}'"
        )
        print(log_message)

    except Exception:
        log_message = (
            f'............................\n'
            f'{__RED__}............................{__RESET__}\n'
            f"{__RED__}An error occurred while dumping the database!{__RESET__}"
        )
        print(log_message)

In [17]:
dump_database(__USER__, __PASSWORD__, __DATABASE__, f"{__DATABASE__}.sql")

............................
............................
............................
Data storing has been completed!

Database property has been dumped to:
 'c:\Users\naufn\Documents\gitHub\Seleksi-2024-Tugas-1\Data Scraping\src\..\..\Data Storing\export\property.sql'
